# Time update for *Cerjan* damped constant density acoustics with forward approximation for $\frac{\partial}{\partial t}$

We show a derivation for the time update expression used for the constant density acoustic solver. You can compare the end result of this derivation in the last equation line below with lines 58-59 in the file ```examples/seismic/acoustic/operators.py```.

**First order numerical derivative (forward)**

$$
\frac{\partial}{\partial t} P(t) = 
    \frac{P(t+\delta t) - P(t)}{\delta t}
$$
<br>

**Second order numerical derivative**

$$
\frac{\partial^2}{\partial t^2} P(t) = 
    \frac{P(t+\delta t) - 2 P(t) + P(t-\delta t)}{\delta t^2}
$$
<br>

**Second order time update**

$$
P(t+\delta t) = \delta t^2 \frac{\partial^2}{\partial t^2} P 
    + 2 P(t) - P(t-\delta t)
$$
<br>

**Damped wave equation**

$$
\begin{aligned}
m \frac{\partial^2}{\partial t^2} P(t) + \eta\ \frac{\partial}{\partial t} P(t) &=
    \nabla^2 P(t) + q \\[10pt]
\frac{\partial^2}{\partial t^2} P(t) &=
    \frac{1}{m} \left( \nabla^2 P(t) + q - \eta \frac{\partial}{\partial t} P(t) \right) 
\end{aligned}
$$

**Time update**

$$
\begin{aligned}
P(t+\delta t) &=
    \frac{\delta t^2}{m} \left\{ \nabla^2 P(t) + q 
        - \frac{\eta}{\delta t} \left[P(t+\delta t) - P(t)\right] \right\} 
    + 2 P(t) - P(t-\delta t) \\[10pt]
\left(1 + \frac{\delta t\ \eta}{m}\right) P(t+\delta t) &= 
    \left\{ 
        \frac{\delta t^2}{m} \left( \nabla^2 P(t) + q \right)
        + \frac{\delta t\ \eta}{m} P(t)
        + 2 P(t)
        - P(t-\delta t)
    \right\} \\[10pt]
P(t+\delta t) &=
    \left( \frac{1}{m+\delta t\ \eta} \right) \left\{ 
        \delta t^2 \left( \nabla^2 P(t) + q \right)
        + \delta t\ \eta P(t)
        + m \left[2 P(t) - P(t-\delta t) \right]
    \right\}
\end{aligned}
$$

**Compare**

<br> Compare the last line in the equation above with [lines 58-59 in examples/seismic/acoustic/operators.py](https://github.com/devitocodes/devito/blob/master/examples/seismic/acoustic/operators.py#L58-L59)